In [3]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pprint
import pyspark
import pyspark.sql.functions as F

from pyspark.sql.functions import col
from pyspark.sql.types import StringType, IntegerType, FloatType, DateType

import utils.data_processing_bronze_table
import utils.data_processing_silver_table
import utils.data_processing_gold_table

In [5]:
# Initialize SparkSession
spark = pyspark.sql.SparkSession.builder \
    .appName("dev") \
    .master("local[*]") \
    .getOrCreate()

# Set log level to ERROR to hide warnings
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/27 19:24:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/09/27 19:24:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
# Bronze / Silver / Gold row counts & schemas
for p in [
    "datamart/bronze/lms/",
    "datamart/silver/loan_daily/",
    "datamart/gold/label_store/",
]:
    print("\n===", p, "===")
    df = spark.read.parquet(p)
    print("rows:", df.count())
    df.printSchema()
    df.show(5, truncate=False)



=== datamart/bronze/lms/ ===


25/09/27 19:25:07 ERROR Executor: Exception in task 0.0 in stage 0.0 (TID 0)
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.util.ThreadUtils$.parmap(ThreadUtils.scala:387)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.readParquetFootersInParallel(ParquetFileFormat.scala:443)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.$anonfun$mergeSchemasInParallel$1(ParquetFileFormat.scala:493)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.$anonfun$mergeSchemasInParallel$1$adapted(ParquetFileFormat.scala:485)
	at org.apache.spark.sql.execution.datasources.SchemaMergeUtils$.$anonfun$mergeSchemasInParallel$2(SchemaMergeUtils.scala:80)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:858)
	at org.apache.spark.

Py4JJavaError: An error occurred while calling o26.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (88c6508da389 executor driver): org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.util.ThreadUtils$.parmap(ThreadUtils.scala:387)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.readParquetFootersInParallel(ParquetFileFormat.scala:443)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.$anonfun$mergeSchemasInParallel$1(ParquetFileFormat.scala:493)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.$anonfun$mergeSchemasInParallel$1$adapted(ParquetFileFormat.scala:485)
	at org.apache.spark.sql.execution.datas

In [7]:
import os
print([f for f in os.listdir("datamart/bronze/lms/")][:10])


['bronze_loan_daily_2023_01_01.csv', 'bronze_loan_daily_2023_02_01.csv', 'bronze_loan_daily_2023_03_01.csv', 'bronze_loan_daily_2023_04_01.csv', 'bronze_loan_daily_2023_05_01.csv', 'bronze_loan_daily_2023_06_01.csv', 'bronze_loan_daily_2023_07_01.csv', 'bronze_loan_daily_2023_08_01.csv', 'bronze_loan_daily_2023_09_01.csv', 'bronze_loan_daily_2023_10_01.csv']


In [8]:
from pyspark.sql import functions as F

# Bronze (CSV)
bronze = (spark.read
          .option("header", "true")
          .option("inferSchema", "true")
          .csv("datamart/bronze/lms/"))
print("bronze rows:", bronze.count()); bronze.printSchema()

# Silver (Parquet)
silver = spark.read.parquet("datamart/silver/loan_daily/")
print("silver rows:", silver.count()); silver.printSchema()

# Gold (Parquet)
gold = spark.read.parquet("datamart/gold/label_store/")
print("gold rows:", gold.count()); gold.printSchema()


bronze rows: 104288
root
 |-- loan_id: string (nullable = true)
 |-- Customer_ID: string (nullable = true)
 |-- loan_start_date: date (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- installment_num: integer (nullable = true)
 |-- loan_amt: integer (nullable = true)
 |-- due_amt: double (nullable = true)
 |-- paid_amt: double (nullable = true)
 |-- overdue_amt: double (nullable = true)
 |-- balance: double (nullable = true)
 |-- snapshot_date: date (nullable = true)



AnalysisException: [UNABLE_TO_INFER_SCHEMA] Unable to infer schema for Parquet. It must be specified manually.

In [9]:
import os, glob

print("Silver dir exists? ", os.path.exists("datamart/silver/loan_daily/"))
print("Silver contents: ", os.listdir("datamart/silver/loan_daily/")[:10])

print("Gold dir exists? ", os.path.exists("datamart/gold/label_store/"))
print("Gold contents: ", os.listdir("datamart/gold/label_store/")[:10])


Silver dir exists?  True
Silver contents:  ['silver_loan_daily_2023_01_01.parquet', 'silver_loan_daily_2023_02_01.parquet', 'silver_loan_daily_2023_03_01.parquet', 'silver_loan_daily_2023_04_01.parquet', 'silver_loan_daily_2023_05_01.parquet', 'silver_loan_daily_2023_06_01.parquet', 'silver_loan_daily_2023_07_01.parquet', 'silver_loan_daily_2023_08_01.parquet', 'silver_loan_daily_2023_09_01.parquet', 'silver_loan_daily_2023_10_01.parquet']
Gold dir exists?  True
Gold contents:  ['gold_label_store_2023_01_01.parquet', 'gold_label_store_2023_02_01.parquet', 'gold_label_store_2023_03_01.parquet', 'gold_label_store_2023_04_01.parquet', 'gold_label_store_2023_05_01.parquet', 'gold_label_store_2023_06_01.parquet', 'gold_label_store_2023_07_01.parquet', 'gold_label_store_2023_08_01.parquet', 'gold_label_store_2023_09_01.parquet', 'gold_label_store_2023_10_01.parquet']


In [10]:
# SILVER
silver = spark.read.parquet("datamart/silver/loan_daily/*.parquet")
print("silver rows:", silver.count()); silver.printSchema()

# GOLD
gold = spark.read.parquet("datamart/gold/label_store/*.parquet")
print("gold rows:", gold.count()); gold.printSchema()


silver rows: 104288
root
 |-- loan_id: string (nullable = true)
 |-- Customer_ID: string (nullable = true)
 |-- loan_start_date: date (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- installment_num: integer (nullable = true)
 |-- loan_amt: float (nullable = true)
 |-- due_amt: float (nullable = true)
 |-- paid_amt: float (nullable = true)
 |-- overdue_amt: float (nullable = true)
 |-- balance: float (nullable = true)
 |-- snapshot_date: date (nullable = true)
 |-- mob: integer (nullable = true)
 |-- installments_missed: integer (nullable = true)
 |-- first_missed_date: date (nullable = true)
 |-- dpd: integer (nullable = true)

gold rows: 8974
root
 |-- loan_id: string (nullable = true)
 |-- Customer_ID: string (nullable = true)
 |-- label: integer (nullable = true)
 |-- label_def: string (nullable = true)
 |-- snapshot_date: date (nullable = true)



In [3]:
import pandas as pd
pd.read_csv("datamart/bronze/lms/bronze_loan_daily_2023_01_01.csv")["snapshot_date"].unique()



array(['2023-01-01'], dtype=object)

In [4]:
pd.read_csv("datamart/bronze/features/clickstream/bronze_clickstream_2023_01_01.csv").head()

,fe_1,fe_2,fe_3,fe_4,fe_5,fe_6,fe_7,fe_8,fe_9,fe_10,...,fe_16,fe_17,fe_18,fe_19,fe_20,Customer_ID,snapshot_date,ingest_dt,snap_ym,source_file
0,63,118,80,121,55,193,111,112,-101,83,...,114,35,85,-73,76,CUS_0x1037,2023-01-01,2023-01-01,2023-01,feature_clickstream.csv
1,-108,182,123,4,-56,27,25,-6,284,222,...,35,130,94,111,75,CUS_0x1069,2023-01-01,2023-01-01,2023-01,feature_clickstream.csv
2,-13,8,87,166,214,-98,215,152,129,139,...,125,-130,354,17,302,CUS_0x114a,2023-01-01,2023-01-01,2023-01,feature_clickstream.csv
3,-85,45,200,89,128,54,76,51,61,139,...,163,37,207,180,118,CUS_0x1184,2023-01-01,2023-01-01,2023-01,feature_clickstream.csv
4,55,120,226,-86,253,97,107,68,103,126,...,159,-26,104,118,184,CUS_0x1297,2023-01-01,2023-01-01,2023-01,feature_clickstream.csv


In [7]:
import glob
len(glob.glob("datamart/bronze/features/clickstream/bronze_clickstream_*.csv"))

24

In [8]:
len(glob.glob("datamart/bronze/lms/bronze_loan_daily_*.csv"))

24

In [1]:
# quick_verify.py
import glob, pandas as pd

def count_files(pattern): return len(glob.glob(pattern))

print("clickstream files:", count_files("datamart/bronze/features/clickstream/bronze_clickstream_*.csv"))
print("attributes files :", count_files("datamart/bronze/features/attributes/bronze_attributes_*.csv"))
print("financials files :", count_files("datamart/bronze/features/financials/bronze_financials_*.csv"))
print("lms files        :", count_files("datamart/bronze/lms/bronze_loan_daily_*.csv"))

# spot-check one month
df = pd.read_csv("datamart/bronze/lms/bronze_loan_daily_2023_06_01.csv")
print("2023-06-01 snapshot unique:", df["snapshot_date"].unique())


clickstream files: 24
attributes files : 24
financials files : 24
lms files        : 24
2023-06-01 snapshot unique: ['2023-06-01']


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ReadSilver").getOrCreate()

# Example: read attributes Silver for Jan 2023
df = spark.read.parquet("datamart/silver/features/attributes/silver_attributes_2023_01_01.parquet")

df.printSchema()
df.show(5, truncate=False)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/03 17:25:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


root
 |-- customer_id: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- snapshot_date: date (nullable = true)
 |-- ingest_dt: date (nullable = true)
 |-- snap_ym: timestamp (nullable = true)
 |-- source_file: string (nullable = true)

+-----------+----------+-------------+----------+-------------------+-----------------------+
|customer_id|occupation|snapshot_date|ingest_dt |snap_ym            |source_file            |
+-----------+----------+-------------+----------+-------------------+-----------------------+
|CUS_0x1037 |ACCOUNTANT|2023-01-01   |2025-10-03|2023-01-01 00:00:00|features_attributes.csv|
|CUS_0x1069 |ACCOUNTANT|2023-01-01   |2025-10-03|2023-01-01 00:00:00|features_attributes.csv|
|CUS_0x114a |DEVELOPER |2023-01-01   |2025-10-03|2023-01-01 00:00:00|features_attributes.csv|
|CUS_0x1184 |LAWYER    |2023-01-01   |2025-10-03|2023-01-01 00:00:00|features_attributes.csv|
|CUS_0x1297 |MANAGER   |2023-01-01   |2025-10-03|2023-01-01 00:00:00|features_attrib

25/10/03 17:25:47 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [3]:
df.select("occupation").distinct().show(100, truncate=False)


+-------------+
|occupation   |
+-------------+
|ACCOUNTANT   |
|DEVELOPER    |
|MECHANIC     |
|ARCHITECT    |
|MEDIA_MANAGER|
|ENGINEER     |
|TEACHER      |
|MUSICIAN     |
|LAWYER       |
|SCIENTIST    |
|WRITER       |
|JOURNALIST   |
|MANAGER      |
|DOCTOR       |
|ENTREPRENEUR |
|NULL         |
+-------------+



In [4]:
spark.stop()

In [1]:
# sanity_financials.py
# Quick checks for Silver financials across a date range.

import os
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

SILVER_FIN_DIR = "datamart/silver/features/financials"   # no trailing slash pls


def month_range(start_str: str, end_str: str):
    start = datetime.strptime(start_str, "%Y-%m-%d")
    end   = datetime.strptime(end_str,   "%Y-%m-%d")
    cur = datetime(start.year, start.month, 1)
    out = []
    while cur <= end:
        out.append(cur.strftime("%Y-%m-%d"))
        cur += relativedelta(months=1)
    return out


def load_month_df(spark, silver_dir, ds):
    token = datetime.strptime(ds, "%Y-%m-%d").strftime("%Y_%m_%d")
    path = os.path.join(silver_dir, f"silver_financials_{token}.parquet")
    return spark.read.parquet(path).withColumn("ds_token", F.lit(ds))


def dq_for_range(spark, silver_dir, start="2023-01-01", end="2024-12-01"):
    months = month_range(start, end)

    # ---------- Load & union ----------
    dfs = []
    for ds in months:
        try:
            dfs.append(load_month_df(spark, silver_dir, ds))
        except Exception as e:
            print(f"[WARN] Missing or unreadable: {ds} ({e})")
    if not dfs:
        print("[ERROR] No Silver financials found.")
        return
    df = dfs[0]
    for d in dfs[1:]:
        df = df.unionByName(d, allowMissingColumns=True)

    print(f"\n[INFO] Loaded months={len(dfs)} rows={df.count()}")

    # ---------- Basic shape & duplicates ----------
    print("\n[COUNT] rows per ds")
    df.groupBy("ds_token").count().orderBy("ds_token").show(50, False)

    print("\n[DUP] duplicate (customer_id, snapshot_date) rows (should be 0):")
    dup = (
        df.groupBy("customer_id", "snapshot_date", "ds_token")
          .count()
          .filter(F.col("count") > 1)
    )
    print(f"duplicate groups: {dup.count()}")

    # ---------- Type compliance & null rates ----------
    expected_cols = [
        "customer_id","snapshot_date","ingest_dt","snap_ym","source_file",
        "annual_income","monthly_inhand_salary","interest_rate",
        "num_of_loan","delay_from_due_date","num_of_delayed_payment",
        "changed_credit_limit","num_credit_inquiries","credit_mix",
        "outstanding_debt","credit_utilization_ratio","credit_history_age",
        "payment_of_min_amount","total_emi_per_month","amount_invested_monthly",
        "payment_behaviour","monthly_balance","type_of_loan"
    ]
    missing = [c for c in expected_cols if c not in df.columns]
    if missing:
        print("[SCHEMA] Missing columns:", missing)

    print("\n[NULL%] key columns (by month)")
    key_cols = ["annual_income","monthly_inhand_salary","outstanding_debt",
                "interest_rate","credit_utilization_ratio","num_of_loan",
                "num_of_delayed_payment","credit_history_age","credit_mix",
                "payment_behaviour","payment_of_min_amount"]
    exprs = [
        (F.avg(F.when(F.col(c).isNull(), 1.0).otherwise(0.0))*100.0).alias(c)
        for c in key_cols if c in df.columns
    ]
    df.groupBy("ds_token").agg(*exprs).orderBy("ds_token").show(50, False)

    # ---------- Numeric ranges & “2dp” checks ----------
    def show_min_max(name):
        if name in df.columns:
            df.select(
                F.min(F.col(name)).alias("min"),
                F.max(F.col(name)).alias("max"),
                F.avg(F.col(name)).alias("mean")
            ).show(truncate=False)

    print("\n[RANGE] interest_rate % (expect ~0..100)")
    show_min_max("interest_rate")
    out_ir = df.filter((F.col("interest_rate") < 0) | (F.col("interest_rate") > 100))
    print(f"interest_rate out-of-range rows: {out_ir.count()}")

    print("\n[RANGE] credit_history_age (months) (expect non-negative & plausible)")
    show_min_max("credit_history_age")
    out_hist = df.filter((F.col("credit_history_age") < 0) | (F.col("credit_history_age") > 1200))
    print(f"credit_history_age implausible rows (>100y or negative): {out_hist.count()}")

    # “2dp” conformance (allow tiny FP error)
    def two_dp_nonconforming(colname):
        return F.avg(
            F.when(F.abs(F.col(colname) - F.round(F.col(colname), 2)) > F.lit(1e-9), 1.0).otherwise(0.0)
        ).alias(colname)

    dp_cols = ["annual_income","monthly_inhand_salary","outstanding_debt",
               "amount_invested_monthly","monthly_balance"]
    print("\n[2DP%] fraction not rounded to 2 d.p. (should be ~0)")
    df.select([two_dp_nonconforming(c) for c in dp_cols if c in df.columns]).show(truncate=False)

    # ---------- Category hygiene ----------
    if "credit_mix" in df.columns:
        print("\n[CREDIT_MIX] distinct values (expect Good/Standard/Bad or NULL)")
        df.groupBy("credit_mix").count().orderBy(F.desc("count")).show(truncate=False)

    if "payment_behaviour" in df.columns:
        print("\n[PAYMENT_BEHAVIOUR] invalid token '!@9#%8' (should be 0)")
        bad_pb = df.filter(F.col("payment_behaviour") == "!@9#%8")
        print(f"rows with bad token: {bad_pb.count()}")

    # ---------- Cross-field consistency ----------
    # Recompute num_of_loan from type_of_loan to verify
    if "type_of_loan" in df.columns and "num_of_loan" in df.columns:
        tol_norm = F.upper(F.col("type_of_loan").cast("string"))
        tol_norm = F.regexp_replace(tol_norm, r"\bAND\b", ",")
        tol_norm = F.regexp_replace(tol_norm, r"NOT SPECIFIED", "")
        tol_norm = F.regexp_replace(tol_norm, r"\s+", " ")
        tol_norm = F.regexp_replace(tol_norm, r",\s*,", ",")
        tol_norm = F.regexp_replace(tol_norm, r"(^\s*,)|(,\s*$)", "")
        df_check = df.withColumn(
            "_num_from_tol",
            F.when(F.length(F.trim(tol_norm)) == 0, F.lit(0))
             .otherwise(F.size(F.split(tol_norm, r"\s*,\s*"))).cast("int")
        )
        mism = df_check.filter(F.col("_num_from_tol") != F.col("num_of_loan"))
        print("\n[CONSISTENCY] num_of_loan vs parsed type_of_loan mismatches:")
        print(f"mismatch rows: {mism.count()}")
        mism.select("ds_token","customer_id","type_of_loan","num_of_loan","_num_from_tol").show(10, False)

    # ---------- Value reasonableness ----------
    nonneg_cols = ["annual_income","monthly_inhand_salary","outstanding_debt",
                   "total_emi_per_month","amount_invested_monthly","monthly_balance",
                   "num_of_loan","num_of_delayed_payment","delay_from_due_date",
                   "num_credit_inquiries","credit_history_age","credit_utilization_ratio"]
    print("\n[NEG%] numeric columns with negatives (should be ~0 except credit_change can be ±)")
    exprs2 = [
        (F.avg(F.when(F.col(c) < 0, 1.0).otherwise(0.0))*100.0).alias(c)
        for c in nonneg_cols if c in df.columns
    ]
    if exprs2:
        df.select(*exprs2).show(truncate=False)

    # ---------- Spot check examples ----------
    print("\n[SAMPLE] interest_rate out-of-range examples")
    out_ir.select("ds_token","customer_id","interest_rate").show(10, False)

    print("\n[SAMPLE] 2dp non-conforming amounts")
    expr_flag = (F.abs(F.col("annual_income") - F.round(F.col("annual_income"), 2)) > F.lit(1e-9))
    if "annual_income" in df.columns:
        df.filter(expr_flag).select("ds_token","customer_id","annual_income").show(10, False)

    print("\n[OK] Sanity checks complete.")


if __name__ == "__main__":
    spark = (
        SparkSession.builder.appName("DQ-Silver-Financials")
        .master("local[*]")
        .getOrCreate()
    )
    spark.sparkContext.setLogLevel("ERROR")

    dq_for_range(
        spark,
        silver_dir=SILVER_FIN_DIR,
        start="2023-01-01",
        end="2024-12-01"
    )

    spark.stop()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/03 19:39:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
                                                                                


[INFO] Loaded months=24 rows=11974

[COUNT] rows per ds


+----------+-----+
|ds_token  |count|
+----------+-----+
|2023-01-01|530  |
|2023-02-01|501  |
|2023-03-01|506  |
|2023-04-01|510  |
|2023-05-01|521  |
|2023-06-01|517  |
|2023-07-01|471  |
|2023-08-01|481  |
|2023-09-01|454  |
|2023-10-01|487  |
|2023-11-01|491  |
|2023-12-01|489  |
|2024-01-01|485  |
|2024-02-01|518  |
|2024-03-01|511  |
|2024-04-01|513  |
|2024-05-01|491  |
|2024-06-01|498  |
|2024-07-01|505  |
|2024-08-01|543  |
|2024-09-01|493  |
|2024-10-01|456  |
|2024-11-01|488  |
|2024-12-01|515  |
+----------+-----+


[DUP] duplicate (customer_id, snapshot_date) rows (should be 0):


duplicate groups: 0

[NULL%] key columns (by month)


+----------+-------------+---------------------+----------------+-------------+------------------------+-----------+----------------------+------------------+------------------+------------------+---------------------+
|ds_token  |annual_income|monthly_inhand_salary|outstanding_debt|interest_rate|credit_utilization_ratio|num_of_loan|num_of_delayed_payment|credit_history_age|credit_mix        |payment_behaviour |payment_of_min_amount|
+----------+-------------+---------------------+----------------+-------------+------------------------+-----------+----------------------+------------------+------------------+------------------+---------------------+
|2023-01-01|0.0          |0.0                  |0.0             |0.0          |0.0                     |0.0        |0.0                   |0.0               |20.754716981132077|7.169811320754717 |0.0                  |
|2023-02-01|0.0          |0.0                  |0.0             |0.0          |0.0                     |0.0        |0.0     

...

+----------+-----+
|credit_mix|count|
+----------+-----+
|Standard  |4287 |
|Good      |2918 |
|NULL      |2505 |
|Bad       |2264 |
+----------+-----+


[PAYMENT_BEHAVIOUR] invalid token '!@9#%8' (should be 0)


rows with bad token: 0

[CONSISTENCY] num_of_loan vs parsed type_of_loan mismatches:


mismatch rows: 2314
+----------+-----------+-----------------------------------------------------------------------------------------------------+-----------+-------------+
|ds_token  |customer_id|type_of_loan                                                                                         |num_of_loan|_num_from_tol|
+----------+-----------+-----------------------------------------------------------------------------------------------------+-----------+-------------+
|2023-01-01|CUS_0x1325 |Not Specified, Personal Loan, Auto Loan, Not Specified, and Debt Consolidation Loan                  |3          |4            |
|2023-01-01|CUS_0x13a8 |NULL                                                                                                 |0          |-1           |
|2023-01-01|CUS_0x1630 |Not Specified, Student Loan, Personal Loan, Not Specified, Not Specified, and Debt Consolidation Loan|3          |4            |
|2023-01-01|CUS_0x169c |NULL                                  

+-------------+---------------------+----------------+-------------------+-----------------------+---------------+-----------+----------------------+-------------------+--------------------+------------------+------------------------+
|annual_income|monthly_inhand_salary|outstanding_debt|total_emi_per_month|amount_invested_monthly|monthly_balance|num_of_loan|num_of_delayed_payment|delay_from_due_date|num_credit_inquiries|credit_history_age|credit_utilization_ratio|
+-------------+---------------------+----------------+-------------------+-----------------------+---------------+-----------+----------------------+-------------------+--------------------+------------------+------------------------+
|0.0          |0.0                  |0.0             |0.0                |0.0                    |0.0            |0.0        |0.0                   |0.7098713880073493 |0.0                 |0.0               |0.0                     |
+-------------+---------------------+----------------+------

+--------+-----------+-------------+
|ds_token|customer_id|annual_income|
+--------+-----------+-------------+
+--------+-----------+-------------+


[OK] Sanity checks complete.


In [2]:
# sanity_financials.py
# Quick checks for Silver financials across a date range (aligned to final Silver rules).
#
# - Loads each month's silver_financials_<YYYY_MM_DD>.parquet and unions them
# - Counts rows/month, duplicate (customer_id, snapshot_date), null rates
# - Range checks: interest_rate in 0..100 (%), credit_history_age months plausible
# - 2 d.p. conformance for money fields
# - Category hygiene for credit_mix and payment_behaviour
# - Consistency: num_of_loan == (raw comma count + 1), with NULL/empty -> 0
# - Reasonableness: negatives for non-negative numeric fields

import os
from datetime import datetime
from dateutil.relativedelta import relativedelta

import pyspark.sql.functions as F
from pyspark.sql import SparkSession

SILVER_FIN_DIR = "datamart/silver/features/financials"   # no trailing slash


def month_range(start_str: str, end_str: str):
    start = datetime.strptime(start_str, "%Y-%m-%d")
    end   = datetime.strptime(end_str,   "%Y-%m-%d")
    cur = datetime(start.year, start.month, 1)
    out = []
    while cur <= end:
        out.append(cur.strftime("%Y-%m-%d"))
        cur += relativedelta(months=1)
    return out


def load_month_df(spark, silver_dir, ds):
    token = datetime.strptime(ds, "%Y-%m-%d").strftime("%Y_%m_%d")
    path = os.path.join(silver_dir, f"silver_financials_{token}.parquet")
    return spark.read.parquet(path).withColumn("ds_token", F.lit(ds))


def dq_for_range(spark, silver_dir, start="2023-01-01", end="2024-12-01"):
    months = month_range(start, end)

    # ---------- Load & union ----------
    dfs = []
    for ds in months:
        try:
            dfs.append(load_month_df(spark, silver_dir, ds))
        except Exception as e:
            print(f"[WARN] Missing or unreadable: {ds} ({e})")
    if not dfs:
        print("[ERROR] No Silver financials found.")
        return

    df = dfs[0]
    for d in dfs[1:]:
        df = df.unionByName(d, allowMissingColumns=True)

    total_rows = df.count()
    print(f"\n[INFO] Loaded months={len(dfs)} rows={total_rows}")

    # ---------- Basic shape & duplicates ----------
    print("\n[COUNT] rows per ds")
    df.groupBy("ds_token").count().orderBy("ds_token").show(50, False)

    print("\n[DUP] duplicate (customer_id, snapshot_date) groups (should be 0):")
    dup = (
        df.groupBy("customer_id", "snapshot_date", "ds_token")
          .count()
          .filter(F.col("count") > 1)
    )
    print(f"duplicate groups: {dup.count()}")

    # ---------- Schema presence ----------
    expected_cols = [
        "customer_id","snapshot_date","ingest_dt","snap_ym","source_file",
        "annual_income","monthly_inhand_salary","interest_rate",
        "num_of_loan","delay_from_due_date","num_of_delayed_payment",
        "changed_credit_limit","num_credit_inquiries","credit_mix",
        "outstanding_debt","credit_utilization_ratio","credit_history_age",
        "payment_of_min_amount","total_emi_per_month","amount_invested_monthly",
        "payment_behaviour","monthly_balance","type_of_loan"
    ]
    missing = [c for c in expected_cols if c not in df.columns]
    if missing:
        print("[SCHEMA] Missing columns:", missing)

    # ---------- Null rates (by month) ----------
    print("\n[NULL%] key columns (by month)")
    key_cols = ["annual_income","monthly_inhand_salary","outstanding_debt",
                "interest_rate","credit_utilization_ratio","num_of_loan",
                "num_of_delayed_payment","credit_history_age","credit_mix",
                "payment_behaviour","payment_of_min_amount"]
    exprs = [
        (F.avg(F.when(F.col(c).isNull(), 1.0).otherwise(0.0))*100.0).alias(c)
        for c in key_cols if c in df.columns
    ]
    if exprs:
        df.groupBy("ds_token").agg(*exprs).orderBy("ds_token").show(50, False)

    # ---------- Ranges ----------
    def show_min_max_mean(name, title=None):
        if name in df.columns:
            print(f"\n[RANGE] {title or name}")
            df.select(
                F.min(F.col(name)).alias("min"),
                F.max(F.col(name)).alias("max"),
                F.avg(F.col(name)).alias("mean")
            ).show(truncate=False)

    # interest_rate in 0..100
    show_min_max_mean("interest_rate", "interest_rate % (expect 0..100)")
    out_ir = df.filter((F.col("interest_rate") < 0) | (F.col("interest_rate") > 100))
    print(f"interest_rate out-of-range rows: {out_ir.count()}")

    # credit_history_age months plausible (>=0, <= 1200 ~ 100y)
    show_min_max_mean("credit_history_age", "credit_history_age months (expect >=0 & <=1200)")
    out_hist = df.filter((F.col("credit_history_age") < 0) | (F.col("credit_history_age") > 1200))
    print(f"credit_history_age implausible rows (>100y or negative): {out_hist.count()}")

    # ---------- “2dp” conformance for money fields ----------
    def two_dp_nonconforming(colname):
        return F.avg(
            F.when(F.abs(F.col(colname) - F.round(F.col(colname), 2)) > F.lit(1e-9), 1.0).otherwise(0.0)
        ).alias(colname)

    dp_cols = ["annual_income","monthly_inhand_salary","outstanding_debt",
               "amount_invested_monthly","monthly_balance"]
    print("\n[2DP%] fraction not rounded to 2 d.p. (should be ~0)")
    df.select([two_dp_nonconforming(c) for c in dp_cols if c in df.columns]).show(truncate=False)

    # ---------- Category hygiene ----------
    if "credit_mix" in df.columns:
        print("\n[CREDIT_MIX] distinct values (expect Good/Standard/Bad or NULL)")
        df.groupBy("credit_mix").count().orderBy(F.desc("count")).show(truncate=False)

    if "payment_behaviour" in df.columns:
        print("\n[PAYMENT_BEHAVIOUR] invalid token '!@9#%8' (should be 0)")
        bad_pb = df.filter(F.col("payment_behaviour") == "!@9#%8")
        print(f"rows with bad token: {bad_pb.count()}")

    # ---------- Consistency: num_of_loan vs (commas + 1) on RAW type_of_loan ----------
    if "type_of_loan" in df.columns and "num_of_loan" in df.columns:
        tol_raw = F.col("type_of_loan").cast("string")
        is_zero = (
            tol_raw.isNull() |
            (F.length(tol_raw) == 0) |
            (F.upper(F.trim(tol_raw)) == F.lit("NULL"))
        )
        comma_count = F.length(tol_raw) - F.length(F.regexp_replace(tol_raw, ",", ""))
        df_check = df.withColumn(
            "_num_from_tol",
            F.when(is_zero, F.lit(0)).otherwise((comma_count + F.lit(1)).cast("int"))
        )
        mism = df_check.filter(F.col("_num_from_tol") != F.col("num_of_loan"))
        print("\n[CONSISTENCY] num_of_loan vs (commas+1) mismatches:")
        print(f"mismatch rows: {mism.count()}")
        mism.select("ds_token","customer_id","type_of_loan","num_of_loan","_num_from_tol").show(10, False)

    # ---------- Reasonableness: non-negative numeric fields ----------
    nonneg_cols = ["annual_income","monthly_inhand_salary","outstanding_debt",
                   "total_emi_per_month","amount_invested_monthly","monthly_balance",
                   "num_of_loan","num_of_delayed_payment","num_credit_inquiries",
                   "credit_history_age","credit_utilization_ratio"]
    print("\n[NEG%] numeric columns with negatives (should be ~0; delay_from_due_date may be ±)")
    exprs2 = [
        (F.avg(F.when(F.col(c) < 0, 1.0).otherwise(0.0))*100.0).alias(c)
        for c in nonneg_cols if c in df.columns
    ]
    if exprs2:
        df.select(*exprs2).show(truncate=False)

    # delay_from_due_date separate (can be negative for early payments)
    if "delay_from_due_date" in df.columns:
        print("\n[DELAY_FROM_DUE_DATE] share negative (early payments)")
        df.select((F.avg(F.when(F.col("delay_from_due_date") < 0, 1.0).otherwise(0.0))*100.0)
                  .alias("neg_%")).show(truncate=False)

    # ---------- Spot samples ----------
    print("\n[SAMPLE] interest_rate out-of-range examples")
    out_ir.select("ds_token","customer_id","interest_rate").show(10, False)

    print("\n[OK] Sanity checks complete.")


if __name__ == "__main__":
    spark = (
        SparkSession.builder.appName("DQ-Silver-Financials")
        .master("local[*]")
        .getOrCreate()
    )
    spark.sparkContext.setLogLevel("ERROR")

    dq_for_range(
        spark,
        silver_dir=SILVER_FIN_DIR,
        start="2023-01-01",
        end="2024-12-01"
    )

    spark.stop()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/05 09:35:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
                                                                                


[INFO] Loaded months=24 rows=11974

[COUNT] rows per ds
+----------+-----+
|ds_token  |count|
+----------+-----+
|2023-01-01|530  |
|2023-02-01|501  |
|2023-03-01|506  |
|2023-04-01|510  |
|2023-05-01|521  |
|2023-06-01|517  |
|2023-07-01|471  |
|2023-08-01|481  |
|2023-09-01|454  |
|2023-10-01|487  |
|2023-11-01|491  |
|2023-12-01|489  |
|2024-01-01|485  |
|2024-02-01|518  |
|2024-03-01|511  |
|2024-04-01|513  |
|2024-05-01|491  |
|2024-06-01|498  |
|2024-07-01|505  |
|2024-08-01|543  |
|2024-09-01|493  |
|2024-10-01|456  |
|2024-11-01|488  |
|2024-12-01|515  |
+----------+-----+


[DUP] duplicate (customer_id, snapshot_date) groups (should be 0):


duplicate groups: 0

[NULL%] key columns (by month)
+----------+-------------+---------------------+----------------+-------------+------------------------+-----------+----------------------+------------------+------------------+------------------+---------------------+
|ds_token  |annual_income|monthly_inhand_salary|outstanding_debt|interest_rate|credit_utilization_ratio|num_of_loan|num_of_delayed_payment|credit_history_age|credit_mix        |payment_behaviour |payment_of_min_amount|
+----------+-------------+---------------------+----------------+-------------+------------------------+-----------+----------------------+------------------+------------------+------------------+---------------------+
|2023-01-01|0.0          |0.0                  |0.0             |0.0          |0.0                     |0.0        |0.0                   |0.0               |20.754716981132077|7.169811320754717 |0.0                  |
|2023-02-01|0.0          |0.0                  |0.0             |0.0    

In [8]:
!pip install -q pyarrow fastparquet



In [7]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.0.1
    Uninstalling pip-25.0.1:
      Successfully uninstalled pip-25.0.1


In [9]:
import pandas as pd

# Example path (adjust to your actual directory)
path = "datamart/silver/loan_daily/silver_loan_daily_2023_01_01.parquet"

df = pd.read_parquet(path)
print(df.shape)
print(df.columns)
df.head(5)


ArrowKeyError: A type extension with name pandas.period already defined

In [10]:
import pandas as pd
path = "datamart/silver/loan_daily/silver_loan_daily_2023_01_01.parquet"
df = pd.read_parquet(path, engine="fastparquet")
print(df.shape)
print(df.columns.tolist())
df.head()


(530, 18)
['loan_id', 'Customer_ID', 'loan_start_date', 'tenure', 'installment_num', 'loan_amt', 'due_amt', 'paid_amt', 'overdue_amt', 'balance', 'snapshot_date', 'ingest_dt', 'snap_ym', 'source_file', 'mob', 'installments_missed', 'first_missed_date', 'dpd']


,loan_id,Customer_ID,loan_start_date,tenure,installment_num,loan_amt,due_amt,paid_amt,overdue_amt,balance,snapshot_date,ingest_dt,snap_ym,source_file,mob,installments_missed,first_missed_date,dpd
0,CUS_0x1037_2023_01_01,CUS_0x1037,2023-01-01,10,0,10000.0,0.0,0.0,0.0,10000.0,2023-01-01,2025-10-03,2023-01-01,lms_loan_daily.csv,0,0,NaT,0
1,CUS_0x1069_2023_01_01,CUS_0x1069,2023-01-01,10,0,10000.0,0.0,0.0,0.0,10000.0,2023-01-01,2025-10-03,2023-01-01,lms_loan_daily.csv,0,0,NaT,0
2,CUS_0x114a_2023_01_01,CUS_0x114a,2023-01-01,10,0,10000.0,0.0,0.0,0.0,10000.0,2023-01-01,2025-10-03,2023-01-01,lms_loan_daily.csv,0,0,NaT,0
3,CUS_0x1184_2023_01_01,CUS_0x1184,2023-01-01,10,0,10000.0,0.0,0.0,0.0,10000.0,2023-01-01,2025-10-03,2023-01-01,lms_loan_daily.csv,0,0,NaT,0
4,CUS_0x1297_2023_01_01,CUS_0x1297,2023-01-01,10,0,10000.0,0.0,0.0,0.0,10000.0,2023-01-01,2025-10-03,2023-01-01,lms_loan_daily.csv,0,0,NaT,0


In [11]:
import os, glob
import pandas as pd

GOLD_DIR = "datamart/gold/label_store"  # adjust if different

# 1) Load all months
paths = sorted(glob.glob(os.path.join(GOLD_DIR, "gold_label_store_*.parquet")))
assert paths, f"No gold files found under {GOLD_DIR}"
dfs = [pd.read_parquet(p, engine="fastparquet") for p in paths]
gold = pd.concat(dfs, ignore_index=True)

# 2) Quick shape & schema
print("Rows, Cols:", gold.shape)
print("Columns:", gold.columns.tolist())
print("\nSample:")
display(gold.head())

# 3) What months & label definitions do we have?
gold["ds_token"] = gold["snapshot_date"].astype(str)  # if already string, this is a no-op
print("\nUnique months:", sorted(gold["ds_token"].unique())[:10], "...",
      sorted(gold["ds_token"].unique())[-3:])
print("\nLabel defs:", gold["label_def"].value_counts())

# 4) Class balance overall and by month
print("\nOverall class balance:")
print(gold["label"].value_counts(normalize=True).mul(100).round(2).rename("pct"))

by_month = (gold.groupby("ds_token")["label"]
            .mean().mul(100).round(2).rename("delinq_pct"))
print("\nDelinquency % by month:")
display(by_month.reset_index().sort_values("ds_token"))

# 5) Uniqueness checks (should be one row per loan per month per label_def)
dups = (gold.groupby(["loan_id","snapshot_date","label_def"])
             .size().reset_index(name="n").query("n > 1"))
print("\nDuplicate (loan_id, snapshot_date, label_def) rows:", len(dups))
if not dups.empty:
    display(dups.head())

# 6) Sanity: label is only 0/1
bad_labels = gold.loc[~gold["label"].isin([0,1])]
print("Non-binary label rows:", len(bad_labels))

# 7) Counts per month (rows, pos rate)
summary = (gold.assign(pos=gold["label"].astype(int))
                .groupby("ds_token")
                .agg(rows=("loan_id","size"),
                     pos_rate=("pos","mean"))
                .assign(pos_rate=lambda d: (d["pos_rate"]*100).round(2))
                .reset_index()
                .sort_values("ds_token"))
print("\nMonthly summary:")
display(summary.head(20))

# 8) Peek a few positive labels
pos = gold.query("label == 1").sample(min(10, (gold['label']==1).sum()), random_state=42)
print("\nPositive label samples:")
display(pos[["loan_id","Customer_ID","snapshot_date","label_def","label"]])


Rows, Cols: (8974, 5)
Columns: ['loan_id', 'Customer_ID', 'label', 'label_def', 'snapshot_date']

Sample:


,loan_id,Customer_ID,label,label_def,snapshot_date
0,CUS_0x1037_2023_01_01,CUS_0x1037,0,30dpd_6mob,2023-07-01
1,CUS_0x1069_2023_01_01,CUS_0x1069,0,30dpd_6mob,2023-07-01
2,CUS_0x114a_2023_01_01,CUS_0x114a,0,30dpd_6mob,2023-07-01
3,CUS_0x1184_2023_01_01,CUS_0x1184,0,30dpd_6mob,2023-07-01
4,CUS_0x1297_2023_01_01,CUS_0x1297,1,30dpd_6mob,2023-07-01



Unique months: ['2023-07-01', '2023-08-01', '2023-09-01', '2023-10-01', '2023-11-01', '2023-12-01', '2024-01-01', '2024-02-01', '2024-03-01', '2024-04-01'] ... ['2024-10-01', '2024-11-01', '2024-12-01']

Label defs: label_def
30dpd_6mob    8974
Name: count, dtype: int64

Overall class balance:
label
0    71.13
1    28.87
Name: pct, dtype: float64

Delinquency % by month:


,ds_token,delinq_pct
0,2023-07-01,26.42
1,2023-08-01,30.74
2,2023-09-01,32.02
3,2023-10-01,26.86
4,2023-11-01,26.68
5,2023-12-01,26.50
6,2024-01-01,29.72
7,2024-02-01,28.90
8,2024-03-01,26.21
9,2024-04-01,25.67



Duplicate (loan_id, snapshot_date, label_def) rows: 0
Non-binary label rows: 0

Monthly summary:


,ds_token,rows,pos_rate
0,2023-07-01,530,26.42
1,2023-08-01,501,30.74
2,2023-09-01,506,32.02
3,2023-10-01,510,26.86
4,2023-11-01,521,26.68
5,2023-12-01,517,26.50
6,2024-01-01,471,29.72
7,2024-02-01,481,28.90
8,2024-03-01,454,26.21
9,2024-04-01,487,25.67



Positive label samples:


,loan_id,Customer_ID,snapshot_date,label_def,label
5544,CUS_0x2b5d_2023_12_01,CUS_0x2b5d,2024-06-01,30dpd_6mob,1
4098,CUS_0x2b29_2023_09_01,CUS_0x2b29,2024-03-01,30dpd_6mob,1
8922,CUS_0xb9da_2024_06_01,CUS_0xb9da,2024-12-01,30dpd_6mob,1
1447,CUS_0xa59_2023_03_01,CUS_0xa59,2023-09-01,30dpd_6mob,1
2447,CUS_0x9be9_2023_05_01,CUS_0x9be9,2023-11-01,30dpd_6mob,1
672,CUS_0x452f_2023_02_01,CUS_0x452f,2023-08-01,30dpd_6mob,1
1039,CUS_0x1324_2023_03_01,CUS_0x1324,2023-09-01,30dpd_6mob,1
7179,CUS_0x6ae7_2024_03_01,CUS_0x6ae7,2024-09-01,30dpd_6mob,1
4282,CUS_0x7694_2023_09_01,CUS_0x7694,2024-03-01,30dpd_6mob,1
618,CUS_0x33aa_2023_02_01,CUS_0x33aa,2023-08-01,30dpd_6mob,1


In [12]:
import os, glob, pandas as pd

GOLD_DIR = "datamart/gold/label_store"
paths = sorted(glob.glob(os.path.join(GOLD_DIR, "gold_label_store_*.parquet")))
gold = pd.concat([pd.read_parquet(p, engine="fastparquet") for p in paths], ignore_index=True)

print("Available label_defs:", gold["label_def"].value_counts().to_dict())

# Filter to MOB=0 labels if they exist
mob0 = gold[gold["label_def"].str.endswith("_0mob", na=False)]
print("MOB=0 label rows:", len(mob0))
mob0.head(10)


Available label_defs: {'30dpd_6mob': 8974}
MOB=0 label rows: 0


,loan_id,Customer_ID,label,label_def,snapshot_date


In [13]:

from pyspark.sql import SparkSession

# 1️⃣ Start Spark session
spark = SparkSession.builder.master("local[*]").appName("CheckSilverFinancials").getOrCreate()

# 2️⃣ Load one month’s silver financials file
silver_path = "datamart/silver/features/financials/silver_financials_2023_01_01.parquet"  # <-- change month if needed
df = spark.read.parquet(silver_path)

# 3️⃣ Show all columns
print("✅ Columns in Silver Financials Table:")
print(df.columns)

# 4️⃣ Preview top 10 rows
print("\n📊 Head (top 10 rows):")
df.show(10, truncate=False)

# 5️⃣ Optional: print schema for data types too
print("\n📐 Schema:")
df.printSchema()

# 6️⃣ Stop Spark (optional if script ends)
spark.stop()

✅ Columns in Silver Financials Table:
['customer_id', 'snapshot_date', 'ingest_dt', 'snap_ym', 'source_file', 'annual_income', 'monthly_inhand_salary', 'interest_rate', 'num_of_loan', 'delay_from_due_date', 'num_of_delayed_payment', 'changed_credit_limit', 'num_credit_inquiries', 'credit_mix', 'outstanding_debt', 'credit_utilization_ratio', 'credit_history_age', 'payment_of_min_amount', 'total_emi_per_month', 'amount_invested_monthly', 'payment_behaviour', 'monthly_balance', 'type_of_loan']

📊 Head (top 10 rows):
+-----------+-------------+-------------------+-------+-----------------------+-------------+---------------------+-------------+-----------+-------------------+----------------------+--------------------+--------------------+----------+----------------+------------------------+------------------+---------------------+-------------------+-----------------------+--------------------------------+---------------+--------------------------------------------------------------------

In [14]:
import pandas as pd

# 1️⃣ Load the Silver financials parquet file
silver_path = "datamart/silver/features/financials/silver_financials_2023_01_01.parquet"  # change the date if needed
df = pd.read_parquet(silver_path)

# 2️⃣ List all columns
print("✅ Columns in Silver Financials Table:")
print(df.columns.tolist())

# 3️⃣ Show the first 5 rows (head)
print("\n📊 Head (first 5 rows):")
print(df.head())

# 4️⃣ (Optional) Show shape for quick overview
print("\n📐 Shape of dataframe:", df.shape)


ArrowKeyError: A type extension with name pandas.period already defined

In [16]:
import os
import glob
import pandas as pd

# 1️⃣ Define the silver financials directory
SILVER_DIR = "datamart/silver/features/financials"

# 2️⃣ Find all monthly parquet files
paths = sorted(glob.glob(os.path.join(SILVER_DIR, "silver_financials_*.parquet")))
print(f"Found {len(paths)} silver financial files")

# 3️⃣ Load them into a single DataFrame (using fastparquet)
silver_financials = pd.concat([pd.read_parquet(p, engine="fastparquet") for p in paths], ignore_index=True)

# 4️⃣ Show available columns
print("✅ Columns in Silver Financials:")
print(silver_financials.columns.tolist())

# 5️⃣ Show shape (rows, cols)
print("\n📐 Shape:", silver_financials.shape)

# 6️⃣ Preview first 10 rows
print("\n📊 Head (first 10 rows):")
print(silver_financials.head(10))

# 7️⃣ (Optional) Null counts for data quality
print("\n🔎 Null counts:")
print(silver_financials.isnull().sum().sort_values(ascending=False))


Found 24 silver financial files
✅ Columns in Silver Financials:
['customer_id', 'snapshot_date', 'ingest_dt', 'snap_ym', 'source_file', 'annual_income', 'monthly_inhand_salary', 'interest_rate', 'num_of_loan', 'delay_from_due_date', 'num_of_delayed_payment', 'changed_credit_limit', 'num_credit_inquiries', 'credit_mix', 'outstanding_debt', 'credit_utilization_ratio', 'credit_history_age', 'payment_of_min_amount', 'total_emi_per_month', 'amount_invested_monthly', 'payment_behaviour', 'monthly_balance', 'type_of_loan']

📐 Shape: (11974, 23)

📊 Head (first 10 rows):
  customer_id snapshot_date  ingest_dt  snap_ym              source_file  \
0  CUS_0x1037    2023-01-01 2025-10-03  2023-01  features_financials.csv   
1  CUS_0x1069    2023-01-01 2025-10-03  2023-01  features_financials.csv   
2  CUS_0x114a    2023-01-01 2025-10-03  2023-01  features_financials.csv   
3  CUS_0x1184    2023-01-01 2025-10-03  2023-01  features_financials.csv   
4  CUS_0x1297    2023-01-01 2025-10-03  2023-01  fe

In [19]:
silver_financials.head(20)

,customer_id,snapshot_date,ingest_dt,snap_ym,source_file,annual_income,monthly_inhand_salary,interest_rate,num_of_loan,delay_from_due_date,...,credit_mix,outstanding_debt,credit_utilization_ratio,credit_history_age,payment_of_min_amount,total_emi_per_month,amount_invested_monthly,payment_behaviour,monthly_balance,type_of_loan
0,CUS_0x1037,2023-01-01,2025-10-03,2023-01,features_financials.csv,15989.09,1086.42,2.00,4,13,...,Good,665.82,40.697699,237,No,33.797021,80.47,Low_spent_Small_value_payments,284.38,"Credit-Builder Loan, Auto Loan, Auto Loan, and..."
1,CUS_0x1069,2023-01-01,2025-10-03,2023-01,features_financials.csv,58637.34,4799.45,10.00,3,9,...,Standard,208.80,25.233144,368,Yes,139.885013,165.21,High_spent_Small_value_payments,434.85,"Personal Loan, Auto Loan, and Not Specified"
2,CUS_0x114a,2023-01-01,2025-10-03,2023-01,features_financials.csv,15305.46,1230.46,2.00,2,14,...,Good,642.42,27.525113,189,No,20.301654,64.78,Low_spent_Small_value_payments,327.97,"Student Loan, and Home Equity Loan"
3,CUS_0x1184,2023-01-01,2025-10-03,2023-01,features_financials.csv,19867.48,1396.62,11.00,3,10,...,Good,707.29,26.689790,392,No,42.606882,23.46,None,313.59,"Student Loan, Mortgage Loan, and Payday Loan"
4,CUS_0x1297,2023-01-01,2025-10-03,2023-01,features_financials.csv,57738.06,4881.51,30.00,9,61,...,Bad,3916.47,25.742143,164,Yes,296.284136,53.82,High_spent_Medium_value_payments,388.05,"Payday Loan, Personal Loan, Payday Loan, Perso..."
5,CUS_0x12fb,2023-01-01,2025-10-03,2023-01,features_financials.csv,26342.91,1949.24,7.00,1,23,...,Standard,397.90,33.211738,264,Yes,57567.000000,48.06,High_spent_Large_value_payments,365.28,Credit-Builder Loan
6,CUS_0x1325,2023-01-01,2025-10-03,2023-01,features_financials.csv,35823.96,2937.33,22.00,5,39,...,Bad,3029.92,31.567171,117,Yes,112.435785,148.52,High_spent_Small_value_payments,292.78,"Not Specified, Personal Loan, Auto Loan, Not S..."
7,CUS_0x1341,2023-01-01,2025-10-03,2023-01,features_financials.csv,15341.90,1035.49,3.00,1,20,...,None,1152.71,24.134405,256,NM,6.981993,81.56,Low_spent_Medium_value_payments,295.00,Debt Consolidation Loan
8,CUS_0x1375,2023-01-01,2025-10-03,2023-01,features_financials.csv,72524.20,5908.68,17.00,3,46,...,Bad,2699.40,23.070388,184,Yes,153.033457,606.46,Low_spent_Medium_value_payments,111.38,"Mortgage Loan, Debt Consolidation Loan, and Ho..."
9,CUS_0x13a8,2023-01-01,2025-10-03,2023-01,features_financials.csv,97059.00,8295.25,9.00,0,9,...,Good,192.99,39.939969,401,No,0.000000,210.62,High_spent_Large_value_payments,858.91,None


In [22]:
silver_financials["delay_from_due_date"].unique()


array([13,  9, 14, 10, 61, 23, 39, 20, 46,  3, 59,  6,  5, 18, 31,  7, 26,
       12, 22, 56,  0, 19, 30, 54, 29,  8, 17, 24,  1, 25, 27, 16, 15, 35,
       36, 44,  2, 11, 42, 28, 62, 47, -3, 21, 53, 40, 60, -5, 43, 45, 65,
       33, 57,  4, 58, -4, 34, 32, 50, 52, 51, 41, 49, 48, 55, 38, -2, 37,
       63, -1, 64, 66, 67], dtype=int32)

In [23]:
silver_financials["delay_from_due_date"].eq(0).sum()


np.int64(141)

In [28]:
silver_financials["delay_from_due_date"].value_counts()



delay_from_due_date
 13    435
 15    435
 14    411
 7     411
 8     402
      ... 
 63      7
-5       5
 64      4
 66      4
 67      3
Name: count, Length: 73, dtype: int64

In [31]:
import os, glob
import pyspark.sql.functions as F
from pyspark.sql import Window

# --- Paths ---
ATTR_DIR  = "datamart/silver/features/attributes"
LMS_DIR   = "datamart/silver/loan_daily"   # your Silver LMS location

# --- Load ALL months of attributes ---
attr_paths = sorted(glob.glob(os.path.join(ATTR_DIR, "silver_attributes_*.parquet")))
attr = spark.read.parquet(*attr_paths)

print("=== SCHEMA (Attributes) ===")
attr.printSchema()

print("=== ROWS & DISTINCT KEYS ===")
n_rows = attr.count()
n_keys = attr.select("customer_id","snapshot_date").distinct().count()
print(f"rows={n_rows:,}  distinct_keys={n_keys:,}")

print("=== ROWS PER SNAPSHOT ===")
attr.groupBy("snapshot_date").count().orderBy("snapshot_date").show(30, False)

print("=== DUPLICATES ON (customer_id, snapshot_date) ===")
dups = (attr
    .groupBy("customer_id","snapshot_date")
    .count().filter("count > 1"))
dups_count = dups.count()
print(f"duplicate key rows = {dups_count}")
if dups_count > 0:
    dups.show(20, False)

print("=== NULLS & CARDINALITY (occupation) ===")
nulls = attr.filter(F.col("occupation").isNull()).count()
distinct_occ = attr.select("occupation").distinct().count()
print(f"occupation_nulls={nulls:,}  occupation_distinct={distinct_occ:,}")

print("=== TOP 20 OCCUPATIONS (by distinct customers per month snapshot) ===")
attr.groupBy("occupation").count().orderBy(F.desc("count")).show(20, False)

print("=== VALUE QUALITY CHECKS (length/pattern) ===")
attr.select(
    F.max(F.length("occupation")).alias("max_len"),
    F.expr("percentile_approx(length(occupation), 0.99)").alias("p99_len")
).show()

print("=== SNAPSHOT COVERAGE VS LABEL STORE (mob=0) ===")
# Load all silver_loan_daily_* and keep mob=0
lms_paths = sorted(glob.glob(os.path.join(LMS_DIR, "silver_loan_daily_*.parquet")))
lms = spark.read.parquet(*lms_paths) \
    .select(F.col("Customer_ID").alias("customer_id"), "snapshot_date", "mob").distinct()

lbl_m0 = lms.filter(F.col("mob") == 0).select("customer_id","snapshot_date").distinct()
attr_keys = attr.select("customer_id","snapshot_date").distinct()

usable = lbl_m0.join(attr_keys, ["customer_id","snapshot_date"], "inner").count()
print(f"labels_mob0={lbl_m0.count():,}   with_attributes={usable:,}   coverage={(usable/max(lbl_m0.count(),1)):.2%}")

print("=== OCCUPATION COVERAGE WITHIN MOB=0 LABELED SET (TOP 15) ===")
(lbl_m0.join(attr, ["customer_id","snapshot_date"], "left")
 .groupBy("occupation")
 .count().orderBy(F.desc("count")).show(15, False))


=== SCHEMA (Attributes) ===
root
 |-- customer_id: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- snapshot_date: date (nullable = true)
 |-- ingest_dt: date (nullable = true)
 |-- snap_ym: timestamp (nullable = true)
 |-- source_file: string (nullable = true)

=== ROWS & DISTINCT KEYS ===


rows=11,974  distinct_keys=11,974
=== ROWS PER SNAPSHOT ===


+-------------+-----+
|snapshot_date|count|
+-------------+-----+
|2023-01-01   |530  |
|2023-02-01   |501  |
|2023-03-01   |506  |
|2023-04-01   |510  |
|2023-05-01   |521  |
|2023-06-01   |517  |
|2023-07-01   |471  |
|2023-08-01   |481  |
|2023-09-01   |454  |
|2023-10-01   |487  |
|2023-11-01   |491  |
|2023-12-01   |489  |
|2024-01-01   |485  |
|2024-02-01   |518  |
|2024-03-01   |511  |
|2024-04-01   |513  |
|2024-05-01   |491  |
|2024-06-01   |498  |
|2024-07-01   |505  |
|2024-08-01   |543  |
|2024-09-01   |493  |
|2024-10-01   |456  |
|2024-11-01   |488  |
|2024-12-01   |515  |
+-------------+-----+

=== DUPLICATES ON (customer_id, snapshot_date) ===


duplicate key rows = 0
=== NULLS & CARDINALITY (occupation) ===


occupation_nulls=846  occupation_distinct=16
=== TOP 20 OCCUPATIONS (by distinct customers per month snapshot) ===
+-------------+-----+
|occupation   |count|
+-------------+-----+
|NULL         |846  |
|LAWYER       |785  |
|ARCHITECT    |766  |
|ACCOUNTANT   |763  |
|ENGINEER     |754  |
|TEACHER      |751  |
|MECHANIC     |750  |
|MEDIA_MANAGER|750  |
|DEVELOPER    |746  |
|SCIENTIST    |746  |
|ENTREPRENEUR |744  |
|JOURNALIST   |741  |
|DOCTOR       |729  |
|MUSICIAN     |709  |
|MANAGER      |700  |
|WRITER       |694  |
+-------------+-----+

=== VALUE QUALITY CHECKS (length/pattern) ===


+-------+-------+
|max_len|p99_len|
+-------+-------+
|     13|     13|
+-------+-------+

=== SNAPSHOT COVERAGE VS LABEL STORE (mob=0) ===


labels_mob0=11,974   with_attributes=11,974   coverage=100.00%
=== OCCUPATION COVERAGE WITHIN MOB=0 LABELED SET (TOP 15) ===


[Stage 57:==================>                                     (8 + 16) / 24]

+-------------+-----+
|occupation   |count|
+-------------+-----+
|NULL         |846  |
|LAWYER       |785  |
|ARCHITECT    |766  |
|ACCOUNTANT   |763  |
|ENGINEER     |754  |
|TEACHER      |751  |
|MECHANIC     |750  |
|MEDIA_MANAGER|750  |
|DEVELOPER    |746  |
|SCIENTIST    |746  |
|ENTREPRENEUR |744  |
|JOURNALIST   |741  |
|DOCTOR       |729  |
|MUSICIAN     |709  |
|MANAGER      |700  |
+-------------+-----+
only showing top 15 rows



In [30]:
# (Re)start Spark if needed
from pyspark.sql import SparkSession

def ensure_spark(app_name="MLE-Inspect"):
    global spark
    try:
        sc = spark.sparkContext  # may fail if spark undefined
        # If Spark exists but is stopped, _jsc.sc().isStopped() will be True
        if sc._jsc.sc().isStopped():
            raise RuntimeError("SparkContext is stopped")
    except Exception:
        spark = (SparkSession.builder
                 .appName(app_name)
                 .master("local[*]")
                 # add any configs you typically use:
                 .config("spark.sql.session.timeZone", "UTC")
                 .getOrCreate())
        spark.sparkContext.setLogLevel("ERROR")
    return spark

spark = ensure_spark()
print("Spark OK:", spark.version)


Spark OK: 3.5.5
